In [105]:
# Set to false if you are not running
# this notebook in Google Colaboratory
run_on_colab = True

if(run_on_colab):
 from google.colab import drive
 # This will prompt for authorization.
 drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [106]:
!pip install music21;
import glob
import pickle
import numpy
import os
import shutil 
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [107]:
cd drive/'My Drive'/'Colab Notebooks'/

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/'
/content/drive/My Drive/Colab Notebooks/Tratamiento de datos


In [ ]:
ls

'Copia de ORIGINAL_LSTM_Music_Generation.ipynb'   note_to_int.pickle
 NetworkInput.pickle                              Tratamientodatos.ipynb
 notes


In [108]:
with open('notes', 'rb') as input2:
    notes = pickle.load(input2)

In [109]:
with open('NetworkInput.pickle', 'rb') as input1:
    Data = pickle.load(input1)

In [ ]:
Data

,Path,Genre,MIDI
0,MIDI_FILES/Country/Brooks_and_Dunn_-_Boot_Scoo...,2,"[771, 418, 418, 771, 418, 771, 418, 418, 771, ..."
1,MIDI_FILES/Country/Brooks_and_Dunn_-_Boot_Scoo...,2,"[779, 779, 344, 779, 238, 734, 734, 221, 765, ..."
2,MIDI_FILES/Country/Brooks_and_Dunn_-_Boot_Scoo...,2,"[607, 602, 719, 719, 657, 742, 673, 772, 772, ..."
3,MIDI_FILES/Country/Brooks_and_Dunn_-_Boot_Scoo...,2,"[673, 772, 719, 719, 657, 673, 720, 772, 772, ..."
4,MIDI_FILES/Country/Brooks_and_Dunn_-_Boot_Scoo...,2,"[734, 719, 673, 792, 771, 418, 418, 771, 418, ..."
...,...,...,...
5571,MIDI_FILES/Jazz/Jazz StandardsDiversen - I Got...,4,"[760, 802, 794, 802, 721, 802, 751, 760, 788, ..."
5572,MIDI_FILES/Jazz/Jazz StandardsDiversen - I Got...,4,"[242, 773, 774, 760, 759, 242, 358, 242, 424, ..."
5573,MIDI_FILES/Jazz/Jazz StandardsDiversen - I Got...,4,"[774, 760, 751, 750, 751, 728, 721, 794, 802, ..."
5574,MIDI_FILES/Jazz/Jazz StandardsDiversen - I Got...,4,"[722, 582, 0, 761, 760, 788, 796, 760, 767, 77..."


In [ ]:
for index, row in Data.iterrows():

'Copia de ORIGINAL_LSTM_Music_Generation.ipynb'   note_to_int.pickle
 NetworkInput.pickle                              Tratamientodatos.ipynb
 notes


In [ ]:
Data[Data['Genre'] == 5]

,Path,Genre,MIDI
1928,MIDI_FILES/Classic/chpn-p15_format0.mid,5,"[788, 742, 793, 744, 120, 793, 793, 793, 728, ..."
1929,MIDI_FILES/Classic/chpn-p15_format0.mid,5,"[788, 742, 793, 744, 794, 116, 793, 793, 793, ..."
1930,MIDI_FILES/Classic/chpn-p15_format0.mid,5,"[793, 788, 427, 743, 487, 116, 338, 65, 788, 7..."
1931,MIDI_FILES/Classic/chpn-p15_format0.mid,5,"[338, 123, 788, 726, 786, 161, 786, 785, 116, ..."
1932,MIDI_FILES/Classic/chpn-p15_format0.mid,5,"[116, 786, 767, 39, 786, 788, 116, 786, 786, 7..."
...,...,...,...
3599,MIDI_FILES/Classic/chpn_op10_e05_format0.mid,5,"[730, 768, 730, 355, 795, 796, 729, 730, 768, ..."
3600,MIDI_FILES/Classic/chpn_op10_e05_format0.mid,5,"[796, 745, 730, 795, 796, 741, 511, 769, 746, ..."
3601,MIDI_FILES/Classic/chpn_op10_e05_format0.mid,5,"[744, 465, 487, 781, 795, 729, 795, 781, 744, ..."
3602,MIDI_FILES/Classic/chpn_op10_e05_format0.mid,5,"[487, 779, 729, 795, 781, 743, 782, 781, 744, ..."


In [ ]:
from itertools import chain
from collections import Counter

[(720, 3313),
 (759, 2882),
 (773, 2637),
 (735, 2580),
 (721, 2551),
 (801, 2449),
 (800, 2414),
 (750, 2260),
 (802, 2187),
 (719, 2085)]

In [ ]:
MIDI_2 = Data[Data['Genre']==2]['MIDI']
MIDI_2_counts = Counter(chain.from_iterable(MIDI_2))
MIDI_2_counts.most_common(10)

In [ ]:
MIDI_4 = Data[Data['Genre']==4]['MIDI']
MIDI_4_counts = Counter(chain.from_iterable(MIDI_4))
MIDI_4_counts.most_common(10)

[(802, 3200),
 (759, 3011),
 (750, 2573),
 (760, 2135),
 (751, 2062),
 (801, 2020),
 (787, 1961),
 (766, 1859),
 (721, 1785),
 (773, 1691)]

In [ ]:
MIDI_1 = Data[Data['Genre']==0]['MIDI']
MIDI_1_counts = Counter(chain.from_iterable(MIDI_1))
MIDI_1_counts.most_common(10)

[(750, 3216),
 (720, 3127),
 (786, 2884),
 (759, 2745),
 (801, 2729),
 (735, 2447),
 (793, 2380),
 (773, 2375),
 (751, 2334),
 (802, 2252)]

In [ ]:
MIDI_3 = Data[Data['Genre']==3]['MIDI']
MIDI_3_counts = Counter(chain.from_iterable(MIDI_3))
MIDI_3_counts.most_common(10)

[(773, 3414),
 (720, 3300),
 (759, 3170),
 (802, 2958),
 (750, 2538),
 (743, 2387),
 (721, 2374),
 (801, 2295),
 (735, 2068),
 (780, 2049)]

In [ ]:
MIDI_5 = Data[Data['Genre']==5]['MIDI']
MIDI_5_counts = Counter(chain.from_iterable(MIDI_5))
MIDI_5_counts.most_common(10)

[(793, 4170),
 (794, 3911),
 (751, 3661),
 (744, 3581),
 (767, 3555),
 (788, 2995),
 (787, 2890),
 (743, 2845),
 (766, 2836),
 (728, 2765)]

In [ ]:
with open('note_to_int.pickle', 'rb') as input2:
    note_to_int = pickle.load(input2)

In [ ]:
note_to_int.values()


dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 21

In [ ]:
for x in note_to_int:
  if
  print(x)


In [ ]:
#para saber que nota es cada una
for x in note_to_int:
  if note_to_int[x] == 766:
    print(x)

E-4


In [ ]:
note_to_int

In [ ]:
n = note.Note('G3', quarterLength=1.5)
n.fullName

'G in octave 3 Dotted Quarter Note'